In [1]:
import pandas                   as pd       
import pyspark
import pyspark.sql
from pyspark.sql                import SparkSession, DataFrame
#from pyspark.sql.Dataframe      import toDF
import pyspark.sql.types        as T
import pyspark.sql.functions    as F  

In [2]:
spark = SparkSession.builder.master('local[*]').appName('risk').getOrCreate()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [3]:
df_data = pd.read_csv('./TD_Expert_Selection_D4_I5_R0.csv')  
df_data['index'] = range(1,len(df_data)+1)
df_data = spark.createDataFrame(df_data)
df_data.show(10, truncate=False)

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+-----+
|d1         |d2         |d3         |d4         |d5         |d6         |d7         |d8         |action|index|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+-----+
|5589.792219|0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |H0    |1    |
|0.0        |5589.792219|0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |H1    |2    |
|0.0        |0.0        |5589.792219|0.0        |0.0        |0.0        |0.0        |0.0        |H0    |3    |
|0.0        |0.0        |0.0        |5589.792219|0.0        |0.0        |0.0        |0.0        |H0    |4    |
|0.0        |0.0        |0.0        |0.0        |5589.792219|0.0        |0.0        |0.0        |H8    |5    |
|2788.98275 |0.0        |0.0        |0.0        |0.0        |5589.792219|0.0        |0.0        |H2    |6    |
|

In [4]:
df_data = df_data.withColumn(
    'action',
    F.substring('action',-1,1).cast(T.IntegerType())
).withColumnRenamed(
    'action', 
    'h'
).withColumn(
    'category',
    F.when(
        F.col('h') < 4, 
        0   # Mutational
    ).when(
        F.col('h') < 6, 
        1 # Ruin-Recreate
    ).when(
        F.col('h') < 10, 
        2#'local-search'
    ).otherwise(
        3#'crossover'
    )
) 
df_data.show(10,truncate=False)

+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---+-----+--------+
|d1         |d2         |d3         |d4         |d5         |d6         |d7         |d8         |h  |index|category|
+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---+-----+--------+
|5589.792219|0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |0  |1    |0       |
|0.0        |5589.792219|0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |1  |2    |0       |
|0.0        |0.0        |5589.792219|0.0        |0.0        |0.0        |0.0        |0.0        |0  |3    |0       |
|0.0        |0.0        |0.0        |5589.792219|0.0        |0.0        |0.0        |0.0        |0  |4    |0       |
|0.0        |0.0        |0.0        |0.0        |5589.792219|0.0        |0.0        |0.0        |8  |5    |2       |
|2788.98275 |0.0        |0.0        |0.0        |0.0        |558

In [5]:
df_index    = df_data.select('index','category').withColumnRenamed('category', 'c8')
df_joined   = df_data.select('index','category').withColumnRenamed('category', 'c8')
for x in range(7): 
    df_temp = df_index.select('index','c8').withColumnRenamed('index', 'new_index').withColumnRenamed('c8', 'curr')

    df_joined = df_joined.join(df_temp,[df_joined.index == (df_temp.new_index + (1 + x))],'left')
    df_joined = df_joined.withColumnRenamed('curr','c{}'.format(7-x)).drop('new_index')
df_joined = df_joined.orderBy(F.col('index').asc())

In [6]:
df_joined.show(20,truncate=False)

+-----+---+----+----+----+----+----+----+----+
|index|c8 |c7  |c6  |c5  |c4  |c3  |c2  |c1  |
+-----+---+----+----+----+----+----+----+----+
|1    |0  |null|null|null|null|null|null|null|
|2    |0  |0   |null|null|null|null|null|null|
|3    |0  |0   |0   |null|null|null|null|null|
|4    |0  |0   |0   |0   |null|null|null|null|
|5    |2  |0   |0   |0   |0   |null|null|null|
|6    |0  |2   |0   |0   |0   |0   |null|null|
|7    |0  |0   |2   |0   |0   |0   |0   |null|
|8    |2  |0   |0   |2   |0   |0   |0   |0   |
|9    |0  |2   |0   |0   |2   |0   |0   |0   |
|10   |0  |0   |2   |0   |0   |2   |0   |0   |
|11   |0  |0   |0   |2   |0   |0   |2   |0   |
|12   |0  |0   |0   |0   |2   |0   |0   |2   |
|13   |0  |0   |0   |0   |0   |2   |0   |0   |
|14   |2  |0   |0   |0   |0   |0   |2   |0   |
|15   |2  |2   |0   |0   |0   |0   |0   |2   |
|16   |0  |2   |2   |0   |0   |0   |0   |0   |
|17   |0  |0   |2   |2   |0   |0   |0   |0   |
|18   |0  |0   |0   |2   |2   |0   |0   |0   |
|19   |0  |0 

In [7]:
df_data = df_data.join(df_joined,['index'],'full')
df_data.show(10,truncate=False)

+-----+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---+--------+---+---+---+---+---+---+---+---+
|index|d1         |d2         |d3         |d4         |d5         |d6         |d7         |d8         |h  |category|c8 |c7 |c6 |c5 |c4 |c3 |c2 |c1 |
+-----+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---+--------+---+---+---+---+---+---+---+---+
|26   |0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |4253.529065|0.0        |0  |0       |0  |0  |0  |0  |0  |2  |2  |0  |
|29   |0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |0.0        |1  |0       |0  |2  |0  |0  |0  |0  |0  |0  |
|474  |73.58608586|0.0        |0.0        |0.0        |0.0        |37.55673741|0.0        |0.0        |4  |1       |1  |0  |0  |0  |0  |1  |2  |2  |
|964  |0.0        |1425.879127|0.0        |0.0        |0.0        |0.0        |0.0        |2.91E-11   |8  

In [8]:
df_data = df_data.orderBy(F.col('index')).select('d1','d2','d3','d4','d5','d6','d7','d8','c1','c2','c3','c4','c5','c6','c7','c8').withColumnRenamed('c8', 'h').na.drop(how='any')
df_data.show(10,truncate=10)

+----------+----------+----------+----------+----------+----------+----------+----------+---+---+---+---+---+---+---+---+
|        d1|        d2|        d3|        d4|        d5|        d6|        d7|        d8| c1| c2| c3| c4| c5| c6| c7|  h|
+----------+----------+----------+----------+----------+----------+----------+----------+---+---+---+---+---+---+---+---+
|4029.43...|       0.0|2788.98275|       0.0|       0.0|       0.0|       0.0|5589.79...|  0|  0|  0|  0|  2|  0|  0|  2|
|       0.0|4029.43...|       0.0|2788.98275|       0.0|       0.0|       0.0|       0.0|  0|  0|  0|  2|  0|  0|  2|  0|
|       0.0|       0.0|4029.43...|       0.0|2788.98275|       0.0|       0.0|       0.0|  0|  0|  2|  0|  0|  2|  0|  0|
|       0.0|       0.0|       0.0|4029.43...|       0.0|2788.98275|       0.0|       0.0|  0|  2|  0|  0|  2|  0|  0|  0|
|       0.0|       0.0|       0.0|       0.0|4029.43...|       0.0|2788.98275|       0.0|  2|  0|  0|  2|  0|  0|  0|  0|
|       0.0|       0.0| 

In [10]:
df_data.toPandas().to_csv('data.csv',index=None)